In [1]:
import csv
import numpy as np
from collections import OrderedDict

In [2]:
filepath = 'data-raw/srs-example.csv'
headers = True

In [3]:
def mean(list):
    return (float(sum(list))/len(list))

class Team(object):
    def __init__(self, name, spreads, opponents):
        self.name = name
        self.spreads = spreads
        self.opponents = opponents

    def set_average_spread(self):
        self.spread = mean(self.spreads)

In [4]:
csvfile = open(filepath, 'r')
gamereader = csv.reader(csvfile, dialect = 'excel')
if headers:
    next(gamereader)

#store all teams in a dict as "team name": team object
# teams = {}
teams = OrderedDict()

#loop through games and construct team objects
for game in gamereader:
    t1 = game[0]
    t2 = game[1]
    # print(t1, t2)
    t1spread = int(game[2]) - int(game[3])
    t2spread = -t1spread

    if t1 in teams:
        teams[t1].spreads.append(t1spread)
        teams[t1].opponents.append(t2)
    else:
        teams[t1] = Team(t1, [t1spread], [t2])

    if t2 in teams:
        teams[t2].spreads.append(t2spread)
        teams[t2].opponents.append(t1)
    else:
        teams[t2] = Team(t2, [t2spread], [t1])

csvfile.close()

#calculate the means
for team in teams.keys():
    teams[team].set_average_spread()

In [5]:
for team in teams.keys():
    print(teams[team].name, teams[team].spread)

Atlanta 4.666666666666667
Boston -11.0
Chicago -9.333333333333334
Dallas -0.6666666666666666
Eugene 9.333333333333334
Fairbanks 7.0


In [33]:
#first matrix with the coefficients of each of the variables
terms0 = []
#seccond matrix with the constant term (-average spread)
solutions0 = []
for team in teams.keys():
    #add in a row for each team
    row = []

    #rating = average spread + average opponent rating
    #-> -average spread = -rating + average opponent rating
    #-> -average spread = -rating + 
    #(number of opponents/1) * (opponent 1 rating+opponent 2 rating...)
    #each row of the matrix describes right side equation
    for opp in teams.keys():
        if opp == teams[team].name:
            row.append(-1)
        elif opp in teams[team].opponents:
            row.append(1.0/len(teams[team].opponents))
        else:
            row.append(0)
    terms0.append(row)

    #each row of this matrix describes the left side of the above equation
    solutions0.append(-teams[team].spread)

In [34]:
print(terms0)

[[-1, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0, 0], [0.3333333333333333, -1, 0, 0, 0.3333333333333333, 0.3333333333333333], [0.3333333333333333, 0, -1, 0.3333333333333333, 0.3333333333333333, 0], [0.3333333333333333, 0, 0.3333333333333333, -1, 0, 0.3333333333333333], [0, 0.3333333333333333, 0.3333333333333333, 0, -1, 0.3333333333333333], [0, 0.3333333333333333, 0, 0.3333333333333333, 0.3333333333333333, -1]]


In [46]:
# np.invert(terms0)
np.linalg.inv(terms0)

array([[-2.00159983e+15, -2.00159983e+15, -2.00159983e+15,
        -2.00159983e+15, -2.00159983e+15, -2.00159983e+15],
       [-2.00159983e+15, -2.00159983e+15, -2.00159983e+15,
        -2.00159983e+15, -2.00159983e+15, -2.00159983e+15],
       [-2.00159983e+15, -2.00159983e+15, -2.00159983e+15,
        -2.00159983e+15, -2.00159983e+15, -2.00159983e+15],
       [-2.00159983e+15, -2.00159983e+15, -2.00159983e+15,
        -2.00159983e+15, -2.00159983e+15, -2.00159983e+15],
       [-2.00159983e+15, -2.00159983e+15, -2.00159983e+15,
        -2.00159983e+15, -2.00159983e+15, -2.00159983e+15],
       [-2.00159983e+15, -2.00159983e+15, -2.00159983e+15,
        -2.00159983e+15, -2.00159983e+15, -2.00159983e+15]])

In [45]:
print(solutions0)

[-4.666666666666667, 11.0, 9.333333333333334, 0.6666666666666666, -9.333333333333334, -7.0]


In [36]:
array1 = np.array(terms0)
array2 = np.array(solutions0)
solutions = np.linalg.solve(array1, array2)
# solutions = np.linalg.lstsq(array1, array2)
print(array1)
print(array2)
print(solutions)

[[-1.          0.33333333  0.33333333  0.33333333  0.          0.        ]
 [ 0.33333333 -1.          0.          0.          0.33333333  0.33333333]
 [ 0.33333333  0.         -1.          0.33333333  0.33333333  0.        ]
 [ 0.33333333  0.          0.33333333 -1.          0.          0.33333333]
 [ 0.          0.33333333  0.33333333  0.         -1.          0.33333333]
 [ 0.          0.33333333  0.          0.33333333  0.33333333 -1.        ]]
[-4.66666667 11.          9.33333333  0.66666667 -9.33333333 -7.        ]
[ 0.57037037 -5.62962963 -6.56296296 -0.0962963   7.83703704  7.7037037 ]


In [37]:
rankings = list(zip([teams[team].name for team in teams.keys()], [solution for solution in solutions]))

In [38]:
#print output
print("Rank\tTeam\tScore")
rankings = list(zip([teams[team].name for team in teams.keys()], [ solution for solution in solutions]))
# print(rankings)
for rank, pair in list(zip(range(1,len(rankings)+1), sorted(rankings, reverse = True, key = lambda x: x[1]))):
    print('{}\t{}\t{:.2f}'.format(rank,pair[0],pair[1]))

Rank	Team	Score
1	Eugene	7.84
2	Fairbanks	7.70
3	Atlanta	0.57
4	Dallas	-0.10
5	Boston	-5.63
6	Chicago	-6.56


In [39]:
for team in teams.keys():
    print(teams[team].name, teams[team].spread)

Atlanta 4.666666666666667
Boston -11.0
Chicago -9.333333333333334
Dallas -0.6666666666666666
Eugene 9.333333333333334
Fairbanks 7.0


In [41]:
np.dot(terms0, solutions)

array([-4.66666667, 11.        ,  9.33333333,  0.66666667, -9.33333333,
       -7.        ])

In [ ]:
np.allclose(np.dot(terms0, solutions), solutions0)

In [42]:
solutions2 = [24.33, -30.67, -24, -4.33, 24, 10.67]

In [43]:
np.dot(terms0, solutions2)

array([-43.99666667,  50.33666667,  38.66666667,   7.99666667,
       -38.66666667, -14.33666667])